In [ ]:
#購入フェーズに絞ったプログラム,考えた戦略を実行するbot

import time 
import numpy
import pandas as pd 
import talib
from binance.client import Client 
import math

#バイナンスAPIを取り扱うインスタンス
API_KEY = ''
SECRET_KEY = ''
binance = Client(API_KEY, SECRET_KEY,{"timeout":20}) #タイムアウト、インターネットの設定が難しい

#取引したい銘柄とトレード条件の設定
ticker = 'BNB'          #取引したい銘柄
currency = 'BUSD'       #USDT建て
symbol = ticker + currency #自動売買のペア文字列
interval = 15         #チャート足（5分足）
trading_amount = 15    #一回の売買で取引する金額（ドル）


kirisute = 1 #切り捨てする数字、Step_Size対策 BNB基準
asset_kirisute = 3

#取引情報を入れるDataFrameの作成
df = pd.DataFrame()

#定期的に処理を実行して価格情報を収集する
while True:

    time.sleep(interval) #インターバルの実装

    #もしもオープンオーダーがないのなら
    orders = binance.get_open_orders()

    if str(orders) == "[]":
        print(orders)
        ticker_info = binance.get_ticker(symbol=symbol)
        lastprice = float(ticker_info['lastPrice']) #現在の仮想通貨価格

        #自分のBUSD所持金
        assets_info = binance.get_asset_balance(asset = "BUSD")
        free_assets1 = float(assets_info["free"])
        #自分のBNB所持金 
        assets_info = binance.get_asset_balance(asset = "BNB")
        free_assets2 = float(assets_info["free"])

        #切り捨て
        lastprice = math.floor(lastprice * 10 ** kirisute) / (10 ** kirisute)
        free_assets1 = math.floor(free_assets1 * 10 ** asset_kirisute) / (10 ** asset_kirisute)
        free_assets2 = math.floor(free_assets2 * 10 ** asset_kirisute) / (10 ** asset_kirisute)

        print(lastprice)
        print(free_assets1)
        print(free_assets2)

        #Marketmake
        #指値の幅を決める、これが利益に関係する
        width = 0.0025 #手数料は0.1%
        top_width = lastprice * (1 + width)
        bottom_width = lastprice * (1 - width)

        #切り捨て処理
        top = math.floor(top * 10 ** kirisute) / (10 ** kirisute)
        bottom = math.floor(bottom_width * 10 ** kirisute) / (10 ** kirisute)

        #売り注文を出す
        sell_potision = free_assets2 
        #order = binance.order_market_sell(symbol=symbol, quantity=sell_potision)  
        sell_message = 'made sell order ' + str(sell_potision) + ticker + ' @' + str(top)
        print(sell_message)
                    
        #買い注文を出す
        buy_position = float(free_assets1) / lastprice 
        #order = binance.order_market_buy(symbol=symbol, quantity=buy_position)  
        buy_message = 'made buy order ' + str(buy_position) + ticker + ' @' + str(bottom)
        print(buy_message)

    else:
        print("オーダーが残っています")
        print(orders)


